#Chapter10.[기본] 소비 데이터 분석과 승객 생존율 예측

#Section 01. [판다스 활용] 카드 소비 데이터 분석
##1. 원본 데이터 소개
##2. 데이터 전처리
### 작업용 파일 생성하기

In [3]:
#from google.colab import drive
#drive.mount('/content/drive')

In [4]:
import pandas as pd
with open('bc_card.txt', mode='rb') as f:
  with open('bc_card_output.txt', mode='w') as w:
    data=f.read()
    print(type(data))
    data=data.decode('cp949')
    print(type(data))
    w.write(data)

<class 'bytes'>
<class 'str'>


In [5]:
with open('bc_card_output.txt', mode='r') as f:
 data=f.read()
len(data)

10235713

In [6]:
data[0:100]

'REG_YYMM\tMEGA_CTY_NO\tMEGA_CTY_NM\tCTY_RGN_NO\tCTY_RGN_NM\tADMI_CTY_NO\tADMI_CTY_NM\tMAIN_BUZ_CODE\tMAIN_BU'

In [7]:
split_data=data.split('\t')
split_data[0:25]

['REG_YYMM',
 'MEGA_CTY_NO',
 'MEGA_CTY_NM',
 'CTY_RGN_NO',
 'CTY_RGN_NM',
 'ADMI_CTY_NO',
 'ADMI_CTY_NM',
 'MAIN_BUZ_CODE',
 'MAIN_BUZ_DESC',
 'TP_GRP_NO',
 'TP_GRP_NM',
 'TP_BUZ_NO',
 'TP_BUZ_NM',
 'CSTMR_GUBUN',
 'CSTMR_MEGA_CTY_NO',
 'CSTMR_MEGA_CTY_NM',
 'CSTMR_CTY_RGN_NO',
 'CSTMR_CTY_RGN_NM',
 'SEX_CTGO_CD',
 'AGE_VAL',
 'FLC',
 'AMT',
 'CNT201906',
 '11',
 '서울특별시']

In [8]:
No_Column=23
len(split_data)%No_Column

3

In [9]:
type(data), type(split_data)

(str, list)

In [10]:
data.find('CNT')

257

In [11]:
data=data[:260]+'\t'+data[260:]
split_data=data.split('\t')
split_data[22],split_data[23]

('CNT', '201906')

In [12]:
len(split_data) % No_Column

4

In [13]:
split_data[23],split_data[23+22]

('201906', '1892201906')

In [14]:
split_data[23+22].find('201906')

4

In [15]:
Cycle= 22
i=1 # 첫 번째와 두 번째 고객 레코드 사이의 cnt와 연도를 구분하기 위해 탭(\t)을 넣으므로
while i < round((len(split_data)-Cycle)/Cycle):
 target_index=split_data[No_Column+Cycle*i].find('201906')
 split_data[No_Column+Cycle*i]=split_data[No_Column+Cycle*i][:target_index]+'\t'+split_data[No_Column+Cycle*i][target_index:]
 i+=1

In [16]:
split_data[23],split_data[23+22]

('201906', '1892\t201906')

In [17]:
a=['a', 'b', 'c', 'd', '1', '2', '3']
print(a)
print()
result1="*****".join(a) # join 함수로 리스트를 문자열로 변환
print(result1)

['a', 'b', 'c', 'd', '1', '2', '3']

a*****b*****c*****d*****1*****2*****3


In [18]:
join_split_data='\t'.join(split_data)
join_split_data[:24]

'REG_YYMM\tMEGA_CTY_NO\tMEG'

In [19]:
joined_splited_data=join_split_data.split('\t')
joined_splited_data[:23]

['REG_YYMM',
 'MEGA_CTY_NO',
 'MEGA_CTY_NM',
 'CTY_RGN_NO',
 'CTY_RGN_NM',
 'ADMI_CTY_NO',
 'ADMI_CTY_NM',
 'MAIN_BUZ_CODE',
 'MAIN_BUZ_DESC',
 'TP_GRP_NO',
 'TP_GRP_NM',
 'TP_BUZ_NO',
 'TP_BUZ_NM',
 'CSTMR_GUBUN',
 'CSTMR_MEGA_CTY_NO',
 'CSTMR_MEGA_CTY_NM',
 'CSTMR_CTY_RGN_NO',
 'CSTMR_CTY_RGN_NM',
 'SEX_CTGO_CD',
 'AGE_VAL',
 'FLC',
 'AMT',
 'CNT']

In [20]:
total=[joined_splited_data[i: i+No_Column] for i in range(0,len(joined_splited_data),No_Column)]
total[1]

['201906',
 '11',
 '서울특별시',
 '1162',
 '관악구',
 '11620585',
 '낙성대동',
 '80',
 '음식',
 '80',
 '일반음식',
 '8006',
 '서양음식',
 '내국인',
 '11',
 '서울특별시',
 '1162',
 '관악구',
 '2',
 '30대',
 '2',
 '26284804',
 '1892']

### 데이터프레임 형식으로 저장하기

In [21]:
total=[joined_splited_data[i: i+No_Column] for i in range(0,len(joined_splited_data),No_Column)]
total[-1]

['201906',
 '11',
 '서울특별시',
 '1135',
 '노원구',
 '11350710',
 '상계9동',
 '20',
 '문화',
 '51',
 '학원',
 '5121',
 '초중고교육기관',
 '내국인',
 '41',
 '경기도',
 '4115',
 '의정부시',
 '1',
 '40대',
 '3',
 '5227500',
 '7']

In [22]:
len(total)

100002

In [23]:
import pandas as pd
total_df=pd.DataFrame(total[1:], columns=total[0])
total_df

,REG_YYMM,MEGA_CTY_NO,MEGA_CTY_NM,CTY_RGN_NO,CTY_RGN_NM,ADMI_CTY_NO,ADMI_CTY_NM,MAIN_BUZ_CODE,MAIN_BUZ_DESC,TP_GRP_NO,...,CSTMR_GUBUN,CSTMR_MEGA_CTY_NO,CSTMR_MEGA_CTY_NM,CSTMR_CTY_RGN_NO,CSTMR_CTY_RGN_NM,SEX_CTGO_CD,AGE_VAL,FLC,AMT,CNT
0,201906,11,서울특별시,1162,관악구,11620585,낙성대동,80,음식,80,...,내국인,11,서울특별시,1162,관악구,2,30대,2,26284804,1892
1,201906,11,서울특별시,1159,동작구,11590560,상도4동,30,생활,40,...,내국인,11,서울특별시,1165,서초구,2,20대,1,109290,18
2,201906,11,서울특별시,1162,관악구,11620595,청룡동,30,생활,83,...,내국인,11,서울특별시,1162,관악구,1,20대,1,268850,52
3,201906,11,서울특별시,1144,마포구,11440660,서교동,80,음식,80,...,내국인,11,서울특별시,1138,은평구,1,20대,1,44174450,1790
4,201906,11,서울특별시,1120,성동구,11200550,사근동,80,음식,80,...,내국인,11,서울특별시,1120,성동구,1,20대,1,60338146,3536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99996,201906,11,서울특별시,1165,서초구,11650520,서초2동,30,생활,40,...,내국인,43,충청북도,4311,청주시,1,50대,4,50600,10
99997,201906,11,서울특별시,1117,용산구,11170520,용산2가동,30,생활,40,...,내국인,11,서울특별시,1162,관악구,1,40대,2,38640,7
99998,201906,11,서울특별시,1156,영등포구,11560535,영등포동,30,생활,40,...,내국인,28,인천광역시,2817,미추홀구,2,30대,2,340590,15
99999,201906,11,서울특별시,1141,서대문구,11410585,신촌동,40,쇼핑,42,...,내국인,44,충청남도,4413,천안시,1,20대,2,117100,3


In [24]:
total_df.to_csv('201906.csv', encoding='utf-8')
bc_card=pd.read_csv('201906.csv',encoding='utf-8', index_col=0)
bc_card.head()


,REG_YYMM,MEGA_CTY_NO,MEGA_CTY_NM,CTY_RGN_NO,CTY_RGN_NM,ADMI_CTY_NO,ADMI_CTY_NM,MAIN_BUZ_CODE,MAIN_BUZ_DESC,TP_GRP_NO,...,CSTMR_GUBUN,CSTMR_MEGA_CTY_NO,CSTMR_MEGA_CTY_NM,CSTMR_CTY_RGN_NO,CSTMR_CTY_RGN_NM,SEX_CTGO_CD,AGE_VAL,FLC,AMT,CNT
0,201906,11,서울특별시,1162,관악구,11620585,낙성대동,80,음식,80,...,내국인,11,서울특별시,1162,관악구,2,30대,2,26284804,1892
1,201906,11,서울특별시,1159,동작구,11590560,상도4동,30,생활,40,...,내국인,11,서울특별시,1165,서초구,2,20대,1,109290,18
2,201906,11,서울특별시,1162,관악구,11620595,청룡동,30,생활,83,...,내국인,11,서울특별시,1162,관악구,1,20대,1,268850,52
3,201906,11,서울특별시,1144,마포구,11440660,서교동,80,음식,80,...,내국인,11,서울특별시,1138,은평구,1,20대,1,44174450,1790
4,201906,11,서울특별시,1120,성동구,11200550,사근동,80,음식,80,...,내국인,11,서울특별시,1120,성동구,1,20대,1,60338146,3536


##3. 데이터 분석
###서울시 거주/비거주 고객의 소비 분석
####① 서울시 거주/비거주 고객 수 구하기

In [25]:
len(bc_card)

100001

In [26]:
index1=bc_card[bc_card['CSTMR_MEGA_CTY_NM'] !='서울특별시'].index
seoul_bc_card=bc_card.drop(index1)
seoul_bc_card.reset_index(drop=True, inplace=True)
len(seoul_bc_card)

54150

In [27]:
index2=bc_card[bc_card['CSTMR_MEGA_CTY_NM']=='서울특별시'].index
nonseoul_bc_card=bc_card.drop(index2)
nonseoul_bc_card.reset_index(drop=True, inplace=True)
len(nonseoul_bc_card)

45851

####② 총 소비액 구하기

In [28]:
print(seoul_bc_card['AMT'].sum(axis=0)) # 서울시 거주 고객의 소비액
print(nonseoul_bc_card['AMT'].sum(axis=0)) # 서울시 비거주 고객의 소비액

119663142676
146587135822


####③ 성별 소비액 구하기

In [29]:
seoulsex=pd.pivot_table(seoul_bc_card, index=['SEX_CTGO_CD'], aggfunc='sum')
seoulsex=seoulsex.drop(['ADMI_CTY_NO','CTY_RGN_NO','TP_GRP_NO','MAIN_BUZ_CODE','REG_YYMM','CNT','CSTMR_CTY_RGN_NO','CSTMR_MEGA_CTY_NO','FLC','MEGA_CTY_NO','TP_BUZ_NO'], axis=1)
seoulsex

C:\Users\linab\AppData\Local\Temp\ipykernel_27384\2667472625.py:1: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  seoulsex=pd.pivot_table(seoul_bc_card, index=['SEX_CTGO_CD'], aggfunc='sum')


,AMT
SEX_CTGO_CD,
1,58128378947
2,61534763729


In [30]:
nonseoulsex=pd.pivot_table(nonseoul_bc_card, index = ['SEX_CTGO_CD'], aggfunc = 'sum')
nonseoulsex=nonseoulsex.drop(['ADMI_CTY_NO','CTY_RGN_NO','TP_GRP_NO','MAIN_BUZ_CODE','REG_YYMM','CNT','CSTMR_CTY_RGN_NO','CSTMR_MEGA_CTY_NO','FLC', 'MEGA_CTY_NO','TP_BUZ_NO'], axis=1)

nonseoulsex

C:\Users\linab\AppData\Local\Temp\ipykernel_27384\3815389678.py:1: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  nonseoulsex=pd.pivot_table(nonseoul_bc_card, index = ['SEX_CTGO_CD'], aggfunc = 'sum')


,AMT
SEX_CTGO_CD,
1,73579570815
2,73007565007


####④ 카드 이용 건수 구하기

In [31]:
print(seoul_bc_card['CNT'].sum(axis=0))
print(nonseoul_bc_card['CNT'].sum(axis=0))

5542462
4950200


###편의점 소비 정보 분석
####① 편의점 소비액 구하기

In [32]:
import pandas as pd
bc_card_cov=bc_card[bc_card['TP_BUZ_NO']==4010] # 4010은 편의점
bc_card_cov['AMT'].sum(axis=0)

7299184098

In [33]:
bc_card_gang_cov=bc_card[(bc_card['TP_BUZ_NO']==4010)&(bc_card['CTY_RGN_NM']=='강남구')]
bc_card_gang_cov['AMT'].sum(axis=0)

707275140

####② 강남구 편의점 소비액 분석하기

In [34]:
import pandas as pd
bc_card_gang=bc_card[bc_card['CTY_RGN_NM']=='강남구']
print(bc_card_gang['AMT'].sum(axis=0)) # 강남구 전체
bc_card_gang_cov=bc_card[(bc_card['TP_BUZ_NO']==4010)&(bc_card['CTY_RGN_NM']=='강남구')]
bc_card_gang_cov['AMT'].sum(axis=0)
print(bc_card_gang_cov['AMT'].sum(axis=0)) # 강남구 편의점 전체

35535648502
707275140


In [35]:
bc_card_gang=bc_card_gang_cov
index3 = bc_card_gang_cov[bc_card_gang_cov['CSTMR_CTY_RGN_NM']=='강남구'].index
bc_card_gang1 = bc_card_gang_cov.drop(index3)
bc_card_gang1.reset_index(drop=True, inplace=True)
bc_card_gang1['AMT'].sum(axis=0)
index4 = bc_card_gang_cov[bc_card_gang_cov['CSTMR_CTY_RGN_NM']!='강남구'].index
bc_card_gang2 = bc_card_gang_cov.drop(index4)
bc_card_gang2.reset_index(drop=True, inplace=True)
bc_card_gang2['AMT'].sum(axis=0)
print(bc_card_gang1['AMT'].sum(axis=0)) # 강남구 거주 고객의 강남구 소비액
print(bc_card_gang2['AMT'].sum(axis=0)) # 강남구 비거주 고객의 강남구 소비액

418895250
288379890


##4. 데이터 분석 응용

In [36]:
import pandas as pd
df=pd.read_csv('bc_card_out2020_03.txt', sep="\t", encoding='utf-8')
df1=pd.DataFrame(df)
df1.head()

,REG_YYMM,MEGA_CTY_NO,MEGA_CTY_NM,CTY_RGN_NO,CTY_RGN_NM,ADMI_CTY_NO,ADMI_CTY_NM,MAIN_BUZ_CODE,MAIN_BUZ_DESC,TP_GRP_NO,...,CSTMR_GUBUN,CSTMR_MEGA_CTY_NO,CSTMR_MEGA_CTY_NM,CSTMR_CTY_RGN_NO,CSTMR_CTY_RGN_NM,SEX_CTGO_CD,AGE_VAL,FLC,AMT,CNT
0,202003,11,서울특별시,1168,강남구,11680750,수서동,30,생활,40,...,내국인,11,서울특별시,1168,강남구,2,20대,1,7927440,1089
1,202003,11,서울특별시,1129,성북구,11290660,길음1동,30,생활,70,...,내국인,11,서울특별시,1129,성북구,1,40대,4,274100,25
2,202003,11,서울특별시,1144,마포구,11440555,아현동,30,생활,40,...,내국인,11,서울특별시,1114,중구,1,60대 이상,5,34395725,808
3,202003,11,서울특별시,1126,중랑구,11260590,상봉2동,30,생활,40,...,내국인,11,서울특별시,1126,중랑구,2,20대,1,31860800,3699
4,202003,11,서울특별시,1168,강남구,11680630,대치4동,30,생활,62,...,내국인,41,경기도,4146,용인시,2,50대,4,2546487,45


###서울시 거주/비거주 고객의 소비액 구하기

In [37]:
import pandas as pd
df=pd.read_csv('bc_card_out2020_03.txt', sep='\t', encoding='utf-8')
df1=pd.DataFrame(df)
df1=df1.drop(['CTY_RGN_NO','ADMI_CTY_NO','ADMI_CTY_NM','TP_GRP_NO',\
 'TP_GRP_NM','CSTMR_GUBUN',\
 ], axis=1)
ind2=df1[df1['CSTMR_MEGA_CTY_NM']!='서울특별시'].index # 서울시 비거주 고객 인덱스화
seoul=df1.drop(ind2)
seoul.reset_index(drop=True, inplace=True)
ind3=df1[df1['CSTMR_MEGA_CTY_NM']=='서울특별시'].index # 서울시 거주 고객만 추출
nonseoul=df1.drop(ind3)
nonseoul.reset_index(drop=True, inplace=True)
print(seoul['AMT'].sum(axis=0)) # 서울시 거주 고객 소비액
print(nonseoul['AMT'].sum(axis=0)) # 서울시 비거주 고객 소비액

1385914569631
1940899349900


In [38]:
seoulsex=pd.pivot_table(seoul, index=['SEX_CTGO_CD'], aggfunc = 'sum') # 1 남성 2 여성
seoulsex=seoulsex.drop(['CNT','CSTMR_CTY_RGN_NO','CSTMR_MEGA_CTY_NO','FLC','MAIN_BUZ_CODE','MEGA_CTY_NO','REG_YYMM','TP_BUZ_NO'], axis=1)
seoulsex

C:\Users\linab\AppData\Local\Temp\ipykernel_27384\1108530800.py:1: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  seoulsex=pd.pivot_table(seoul, index=['SEX_CTGO_CD'], aggfunc = 'sum') # 1 남성 2 여성


,AMT
SEX_CTGO_CD,
1,682678945342
2,703235624289


###거주지 소재 편의점 소비액 구하기

In [39]:
import pandas as pd
seoul_cov = seoul[seoul['TP_BUZ_NO']==4010] # 편의점
seoul_cov['CNT'].sum(axis=0)
seoul_cov['AMT'].sum(axis=0)

66513977146

In [40]:
import pandas as pd
seoul=seoul_cov
index1 = seoul[seoul['CTY_RGN_NM'] != seoul['CSTMR_CTY_RGN_NM']].index
resid = seoul.drop(index1)
resid.reset_index(drop=True, inplace=True)
index2 = seoul[seoul['CTY_RGN_NM'] == seoul['CSTMR_CTY_RGN_NM']].index
nonresid = seoul.drop(index2)
nonresid.reset_index(drop=True, inplace=True)
print(resid['AMT'].sum(axis=0)) # 거주지 내 소비액
print(nonresid['AMT'].sum(axis=0)) # 거주지 외 소비액


44184614834
22329362312


### **Section02_승객 생존율 예측**

##1. 분석 준비

In [41]:
!pip install pandas

In [42]:
import pandas as pd
url='https://raw.githubusercontent.com/sehakflower/data/main/titanic.csv'
titanic_df= pd.read_csv(url,sep='\t')

# 컬럼명을 소문자로 바꾸기
new_columns=['passengerId', 'survived', 'pclass', 'name', 'sex', 'age', 'sibsp', 'parch',
'ticket', 'fare', 'cabin', 'embarked']
titanic_df.columns=new_columns

# 필요한 컬럼만 가져오기
titanic_df1=titanic_df[['survived', 'pclass', 'name', 'sex', 'age', 'sibsp', 'parch', 'fare']]

# 1. sex열을 gender로 바꾸고 여성은 0, 남성은 1로 바꾸기
tmp=[]
for each in titanic_df1['sex']:
  if each=='female':
    tmp.append(0)
  else:
    tmp.append(1)

titanic_df1['gender']=tmp
titanic_df1.drop(columns='sex', inplace=True) # sex 열 삭제하기

# 2. 호칭을 숫자형으로 바꾸기
condition=lambda x: x.split(',')[1].split('.')[0].strip()
titanic_df1['title']=titanic_df1['name'].map(condition) # name에서 호칭만 가져오기
Special=['Master', 'Don', 'Rev'] # 3개 호칭을 title 열의 Special로 대체
for each in Special:
  titanic_df1['title']=titanic_df1['title'].replace(each, 'Special')

titanic_df1=titanic_df1.drop('name', axis=1) # 사용하지 않는 name 열 삭제하기

# 숫자형으로 만드는 함수 만들기
def convert_title(x):
  if x=="Special":
    return 1
  else:
    return 0

titanic_df1["special_title"]=titanic_df1["title"].apply(convert_title)
# special_title 열 만들어서 special은 1, 나머지는 0으로 바꾸기
titanic_df1.drop('title', axis=1, inplace=True) # title 열 삭제하기
# 3. sibpar 열 만들어 동반자 수 더한 값 넣기
titanic_df1['sibpar']=titanic_df1['sibsp']+titanic_df1['parch']
titanic_df1.drop(['sibsp','parch'], axis =1, inplace=True)
# 사용하지 않는 sibsp, parch 열 삭제하기
# 4. avgfare 열 만들어 1인당 평균 탑승 요금 넣기
titanic_df1['avgfare']=titanic_df1['fare']/titanic_df1['sibpar']
titanic_df1['n_family']=titanic_df1['sibpar']+1
# n_family 열 만들어 동반자에 1을 더하여 0으로 나누어지지 않도록 함
titanic_df1['avgfare']=titanic_df1['fare']/titanic_df1['n_family'] # 1인당 평균 탑승 요금
titanic_df1=titanic_df1.drop(['fare','sibpar'], axis=1) # 필요 없는 열 삭제하기
# 5. 최종 분석 데이터 컬럼명만 추출하기
titanic_df1.rename(columns={'gender':'sex', 'special_title':'title', 'avgfare':'fare', 'n_family':'num_family'}, inplace=True) # 컬럼명 변경하기
titanic_df1=titanic_df1[['survived', 'pclass', 'sex', 'age', 'title', 'fare', 'num_family']]
# 컬럼 순서 다시 정하기
titanic_df1=titanic_df1.dropna()
# age에는 NaN 값이 있으므로 이를 삭제한 데이터를 분석에 사용
titanic_df1

C:\Users\linab\AppData\Local\Temp\ipykernel_27384\1157815611.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  titanic_df1['gender']=tmp
C:\Users\linab\AppData\Local\Temp\ipykernel_27384\1157815611.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  titanic_df1.drop(columns='sex', inplace=True) # sex 열 삭제하기
C:\Users\linab\AppData\Local\Temp\ipykernel_27384\1157815611.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

,survived,pclass,sex,age,title,fare,num_family
0,0,3,1,22.0,0,3.625000,2
1,1,1,0,38.0,0,35.641650,2
2,1,3,0,26.0,0,7.925000,1
3,1,1,0,35.0,0,26.550000,2
4,0,3,1,35.0,0,8.050000,1
...,...,...,...,...,...,...,...
150,0,2,1,51.0,1,12.525000,1
151,1,1,0,22.0,0,33.300000,2
152,0,3,1,55.5,0,8.050000,1
153,0,3,1,40.5,0,4.833333,3


In [43]:
raw=titanic_df1
np_raw=raw.values
type(np_raw)

numpy.ndarray

##2. 의사결정나무

In [44]:
!pip3 install -U scikit-learn

     ---------------------------------------- 9.3/9.3 MB 10.6 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.1
    Uninstalling scikit-learn-1.2.1:
      Successfully uninstalled scikit-learn-1.2.1


In [45]:
from sklearn.tree import DecisionTreeClassifier
model=DecisionTreeClassifier(criterion='entropy',max_depth=3,min_samples_leaf=5).fit(X_train, y_train)
model.fit(X_train, y_train)
DecisionTreeClassifier(criterion='entropy', max_depth=3, min_samples_leaf=5)
print('Score:{}'.format(model.score(X_train, y_train)))
print('Score:{}'.format(model.score(X_test, y_test)))

NameError: name 'X_train' is not defined

In [ ]:
from sklearn.tree import export_graphviz
export_graphviz(
 model,
 out_file="titanic.dot",
 feature_names=['pclass', 'sex', 'age', 'title', 'fare', 'num_family'],
 class_names=['0','1'],
 rounded=True,
 filled=True
)
import graphviz
with open("titanic.dot") as f:
 dot_graph=f.read()
dot=graphviz.Source(dot_graph)
dot.format='png'
dot.render(filename='titanic_tree', directory='image/decision_trees', cleanup=True)
dot

In [ ]:
from sklearn.metrics import accuracy_score
y_pred=model.predict(X_test)
print("Test Accuracy is ", accuracy_score(y_test, y_pred)*100)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

In [ ]:
feature_names=['pclass', 'sex', 'age', 'title', 'fare', 'num_family']
Tom=[1,1,33,1,50,4]
Jane=[2,0,50,0,8,1]

In [ ]:
model.predict_proba([Tom])

In [ ]:
model.predict_proba([Jane])

###표준화한 데이터 이용하기

In [ ]:
!pip install seaborn

In [ ]:
import seaborn as sns
sns.boxplot(data=titanic_df1[['pclass', 'sex', 'age', 'title', 'fare', 'num_family']])

In [ ]:
X=titanic_df1
from sklearn.preprocessing import MinMaxScaler, StandardScaler
MMS=MinMaxScaler()
SS=StandardScaler()
SS.fit(X)
MMS.fit(X)
X_ss=SS.transform(X)
X_mms=MMS.transform(X)
X_ss_pd=pd.DataFrame(X_ss, columns=X.columns)
X_mms_pd=pd.DataFrame(X_mms, columns=X.columns)
X_ss_pd.head()

In [ ]:
X_mms_pd.head()

In [ ]:
sns.boxplot(data=X_mms_pd[['pclass','sex','age','title','fare','num_family']])

In [ ]:
sns.boxplot(data=X_ss_pd[['pclass', 'sex', 'age', 'title', 'fare', 'num_family']])

In [ ]:
y=raw['survived']
X=raw.drop(['survived'], axis=1)
X.head()

In [ ]:
y.head()

In [ ]:
import numpy as np
np.unique(y_train, return_counts=True)

In [ ]:
log_reg=LogisticRegression(random_state=13, solver='liblinear', C=10.)
log_reg.fit(X_train, y_train)

In [ ]:
pred=log_reg.predict(X_test)
accuracy_score(y_test, pred)

In [ ]:
pred=log_reg.predict(X_test)
accuracy_score(y_test, pred)

In [ ]:
pred=log_reg.predict(X_test)
accuracy_score(y_test, pred)

In [ ]:
pred=log_reg.predict(X_test)
accuracy_score(y_test, pred)

In [ ]:
log_reg.coef_

### **Section03_승객 생존율 예측2**

##1. 머신러닝 기법에 적용하기


In [ ]:
import pandas as pd
titanic_url='https://github.com/sehakflower/data/blob/main/titanic_1309.xlsx?raw=true'
titanic=pd.read_excel(titanic_url,sheet_name='total')
train_1000=titanic.iloc[:1000]
test_309=titanic.iloc[1000:]
train_1000.drop(['boat', 'body', 'home.dest'], axis=1, inplace=True)
test_309.drop(['boat', 'body', 'home.dest'], axis=1, inplace=True)
train_df=train_1000
test_df=test_309

# 1. name 열을 5개로 구분하고 숫자형 데이터로 바꾸기
total=[train_df, test_df]
titles={'Mr':1, 'Miss':2, 'Mrs':3, 'Master':4, 'Special':5} # 호칭에 따라 숫자로 표시하기
for dataset in total:
 dataset['title']=dataset.name.str.extract('([A-Za-z]+)\.', expand=False)
 dataset['title']=dataset['title'].replace(['Lady', 'Countess', 'Capt', 'Col',
'Don', 'Dr', 'Major', 'Rev', 'Sir',
'Jonkheer', 'Dona'], 'Special')
 dataset['title']=dataset['title'].replace('Mlle', 'Miss')
 dataset['title']=dataset['title'].replace('Ms', 'Miss')
 dataset['title']=dataset['title'].replace('Mme', 'Mrs')
 dataset['title']=dataset['title'].map(titles) # 타이틀을 숫자로 변경
 dataset['title']=dataset['title'].fillna(0) # NaN을 0으로 변경
train_df=train_df.drop(['name'], axis=1) # 필요 없는 name 열 삭제
test_df=test_df.drop(['name'], axis=1) # 필요 없는 name 열 삭제

# 2.age 열의 빈 값을 호칭별 중앙값으로 바꾸기
train_df['age'].fillna(train_df.groupby('title')['age'].transform('median'), inplace=True)
test_df['age'].fillna(test_df.groupby('title')['age'].transform('median'), inplace=True)
data=[train_df, test_df]
for dataset1 in data:
 dataset1['age']=dataset1['age'].astype(int)
 dataset1.loc[ dataset1['age']<=11, 'age']=0
 dataset1.loc[(dataset1['age']>11)&(dataset1['age']<=18), 'age']=1
 dataset1.loc[(dataset1['age']>18)&(dataset1['age']<=22), 'age']=2
 dataset1.loc[(dataset1['age']>22)&(dataset1['age']<=27), 'age']=3
 dataset1.loc[(dataset1['age']>27)&(dataset1['age']<=33), 'age']=4
 dataset1.loc[(dataset1['age']>33)&(dataset1['age']<=40), 'age']=5
 dataset1.loc[(dataset1['age']>40)&(dataset1['age']<=66), 'age']=6
 dataset1.loc[ dataset1['age']>66, 'age']=6
# 3. sex 열에서 'male':0, 'female':1로 바꾸기
data=[train_df, test_df]
# 일부 전처리된 train_df, test_df를 다시 업데이트해서 더한 후 숫자로 바꾸기 위함
sex_mapping={'male':0, 'female':1}
for dataset in data:
 dataset['sex']=dataset['sex'].map(sex_mapping)
# 4. embarked 열
for dataset in data:
 dataset['embarked']=dataset['embarked'].fillna('S')
embarked_mapping={'S':0, 'C':1, 'Q':2}
for dataset in data:
 dataset['embarked']=dataset['embarked'].map(embarked_mapping)
# 5. sibsp, parch 열
for dataset in data:
 dataset['sibpar']=dataset['sibsp']+dataset['parch']
 dataset.loc[dataset['sibpar']>0, 'n_alone']=0
 dataset.loc[dataset['sibpar']==0, 'n_alone']=1
 dataset['n_alone']=dataset['n_alone'].astype(int)
# 6. fare 열, 개인별 요금 나타내는 fare_person 열 추가하기
train_df["fare"].fillna(train_df.groupby('pclass')['fare'].transform('median'),
inplace=True)
test_df["fare"].fillna(test_df.groupby('pclass')['fare'].transform('median'),
inplace=True)

for dataset in data:
 dataset.loc[ dataset['fare']<=20, 'fare']=1
 dataset.loc[(dataset['fare']>20)&(dataset['fare']<=30), 'fare']=2
 dataset.loc[(dataset['fare']>30)&(dataset['fare']<=50), 'fare']=3
 dataset.loc[(dataset['fare']>50)&(dataset['fare']<=100), 'fare']=4
 dataset.loc[ dataset['fare']>100, 'fare']=5
for dataset1 in data:
 dataset1['fare_person']=dataset1['fare']/(dataset1['sibpar']+1)
 dataset1['fare_person']=dataset1['fare_person'].astype(int)
# 7. 최종 데이터
X_columns=['pclass', 'sex', 'age', 'embarked', 'title', 'sibpar', 'n_alone', 'fare_person']
y_column=['survived']
X_train=train_df[X_columns]
y_train=train_df[y_column]
X_test=test_df[X_columns]
y_test=test_df[y_column]

###필요한 라이브러리

In [ ]:
import numpy as np # 선형대수
import pandas as pd # 데이터 처리
# 머신러닝 알고리즘
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB
# 시각화
import seaborn as sns
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import style

### 의사결정나무

In [ ]:
decision_tree=DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)
Y_pred=decision_tree.predict(X_test)
train_acc_decision_tree=round(decision_tree.score(X_train, y_train)*100, 2)
test_acc_decision_tree=round(decision_tree.score(X_test, y_test)*100, 2)
train_acc_decision_tree,test_acc_decision_tree

### 랜덤 포레스트

In [ ]:
random_forest=RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, y_train)
Y_prediction=random_forest.predict(X_test)
random_forest.score(X_train, y_train)
train_acc_random_forest=round(random_forest.score(X_train, y_train)*100, 2)
test_acc_random_forest=round(random_forest.score(X_test, y_test)*100, 2)
train_acc_random_forest,test_acc_random_forest

### 로지스틱 회귀분석

In [ ]:
log_reg=LogisticRegression()
log_reg.fit(X_train, y_train)
Y_pred=log_reg.predict(X_test)
train_acc_log=round(log_reg.score(X_train, y_train)*100, 2)
test_acc_log=round(log_reg.score(X_test, y_test)*100, 2)
train_acc_log, test_acc_log

### 나이브 베이지안

In [ ]:
gaussian=GaussianNB()
gaussian.fit(X_train, y_train)
Y_pred=gaussian.predict(X_test)
train_acc_gaussian=round(gaussian.score(X_train, y_train)*100, 2)
test_acc_gaussian=round(gaussian.score(X_test, y_test)*100, 2)
train_acc_gaussian,test_acc_gaussian

###K-Nearest Neighbor

In [ ]:
knn=KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
Y_pred=knn.predict(X_test)
train_acc_knn=round(knn.score(X_train, y_train)*100, 2)
test_acc_knn=round(knn.score(X_test, y_test)*100, 2)
train_acc_knn,test_acc_knn

### SVM 모델

In [ ]:
svc=LinearSVC()
svc.fit(X_train, y_train)
Y_pred=svc.predict(X_test)
train_acc_svc=round(svc.score(X_train, y_train)*100, 2)
test_acc_svc=round(svc.score(X_test, y_test)*100, 2)
train_acc_svc,test_acc_svc

### 퍼셉트론 모델

In [ ]:
perceptron=Perceptron(max_iter=5)
perceptron.fit(X_train, y_train)
Y_pred=perceptron.predict(X_test)
train_acc_perceptron=round(perceptron.score(X_train, y_train)*100, 2)
test_acc_perceptron=round(perceptron.score(X_test, y_test)*100, 2)
train_acc_perceptron,test_acc_perceptron

### 확률적 경사하강법 방법

In [ ]:
sgd=linear_model.SGDClassifier(max_iter=5, tol=None)
sgd.fit(X_train, y_train)
Y_pred=sgd.predict(X_test)
sgd.score(X_train, y_train)
train_acc_sgd=round(sgd.score(X_train, y_train)*100, 2)
test_acc_sgd=round(sgd.score(X_test, y_test)*100, 2)
train_acc_sgd,test_acc_sgd

## 2. 모델 비교
###모델의 정확도 비교

In [ ]:
results = pd.DataFrame({
 'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 'Random Forest','Naive Bayes', 'Perceptron', 'Stochastic Gradient Decent', 'Decision Tree'],
 'train_Score': [train_acc_svc, train_acc_knn, train_acc_log,train_acc_random_forest,
train_acc_gaussian, train_acc_perceptron, train_acc_sgd, train_acc_decision_tree],
 'test_Score': [test_acc_svc, test_acc_knn,test_acc_log, test_acc_random_forest, test_acc_gaussian, test_acc_perceptron, test_acc_sgd, test_acc_decision_tree]})
result_df=results.sort_values(by='train_Score', ascending=False)
result_df=result_df.set_index('Model')
result_df.head(10)

###랜덤 포레스트 모델의 교차검증 실습

In [ ]:
from sklearn.model_selection import cross_val_score
rf=RandomForestClassifier(n_estimators=100)
scores=cross_val_score(rf, X_train, y_train, cv=10, scoring="accuracy")
print("Scores:", scores)
print("Mean:", scores.mean())
print("Standard Deviation:", scores.std())